In [58]:
import sys
sys.path.append('..')

In [59]:
import pandas as pd
from continuous_BCQ.utils import ReplayBuffer

In [60]:
df = pd.read_csv("df_thickener1_20220522.csv")

In [61]:
df

,time,feed_c,out_c,feed_f,out_f,pressure,out_f_setting,out_c_setting,reward,error,sum_error,done
0,2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,133.133333,52.0,-210.152231,-14.496628,-14.496628,0
1,2021-05-29 11:55:00,18.514265,68.554742,594.701354,54.951627,36.228900,141.116667,52.0,-274.059494,-16.554742,-23.803056,0
2,2021-05-29 12:00:00,20.828083,65.206491,600.521913,137.307769,49.096255,135.000000,52.0,-174.411393,-13.206491,-25.108019,0
3,2021-05-29 12:05:00,18.641244,65.202654,619.654978,133.066213,42.642862,131.366667,52.0,-174.310072,-13.202654,-25.756663,0
4,2021-05-29 12:10:00,21.932207,65.288382,631.921162,123.592218,43.254578,130.000000,52.0,-176.581087,-13.288382,-26.166713,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12004,2022-05-07 09:50:00,33.020739,59.366599,601.963274,146.562356,51.578216,140.399994,62.0,-6.934803,2.633401,4.412321,0
12005,2022-05-07 09:55:00,29.091299,60.463238,679.524404,136.653616,50.474077,140.399994,62.0,-2.361637,1.536762,3.742922,0
12006,2022-05-07 10:00:00,26.692466,59.308527,601.825201,157.848585,51.421173,140.399994,62.0,-7.244026,2.691473,4.562934,0
12007,2022-05-07 10:05:00,28.122638,59.329402,612.676073,148.789828,51.545893,140.399994,62.0,-7.132092,2.670598,4.952065,0


In [63]:
order = ["feed_c", "out_c", "feed_f", "out_f", "out_f_setting", "pressure", "error", "sum_error", "out_c_setting", "reward", "out_f_setting", "done"]
df = df[order]

In [64]:
df

,feed_c,out_c,feed_f,out_f,out_f_setting,pressure,error,sum_error,out_c_setting,reward,out_f_setting,done
0,18.687975,66.496628,587.393447,47.888910,133.133333,42.066571,-14.496628,-14.496628,52.0,-210.152231,133.133333,0
1,18.514265,68.554742,594.701354,54.951627,141.116667,36.228900,-16.554742,-23.803056,52.0,-274.059494,141.116667,0
2,20.828083,65.206491,600.521913,137.307769,135.000000,49.096255,-13.206491,-25.108019,52.0,-174.411393,135.000000,0
3,18.641244,65.202654,619.654978,133.066213,131.366667,42.642862,-13.202654,-25.756663,52.0,-174.310072,131.366667,0
4,21.932207,65.288382,631.921162,123.592218,130.000000,43.254578,-13.288382,-26.166713,52.0,-176.581087,130.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12004,33.020739,59.366599,601.963274,146.562356,140.399994,51.578216,2.633401,4.412321,62.0,-6.934803,140.399994,0
12005,29.091299,60.463238,679.524404,136.653616,140.399994,50.474077,1.536762,3.742922,62.0,-2.361637,140.399994,0
12006,26.692466,59.308527,601.825201,157.848585,140.399994,51.421173,2.691473,4.562934,62.0,-7.244026,140.399994,0
12007,28.122638,59.329402,612.676073,148.789828,140.399994,51.545893,2.670598,4.952065,62.0,-7.132092,140.399994,0


In [101]:
#"name": ["feed_c", "out_c", "feed_f", "out_f", "out_f_setting", "pressure", "reward"]

In [86]:
df_1 = df

In [87]:
df_1

,feed_c,out_c,feed_f,out_f,out_f_setting,pressure,error,sum_error,out_c_setting,reward,out_f_setting,done
0,18.687975,66.496628,587.393447,47.888910,133.133333,42.066571,-14.496628,-14.496628,52.0,-210.152231,133.133333,0
1,18.514265,68.554742,594.701354,54.951627,141.116667,36.228900,-16.554742,-23.803056,52.0,-274.059494,141.116667,0
2,20.828083,65.206491,600.521913,137.307769,135.000000,49.096255,-13.206491,-25.108019,52.0,-174.411393,135.000000,0
3,18.641244,65.202654,619.654978,133.066213,131.366667,42.642862,-13.202654,-25.756663,52.0,-174.310072,131.366667,0
4,21.932207,65.288382,631.921162,123.592218,130.000000,43.254578,-13.288382,-26.166713,52.0,-176.581087,130.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12004,33.020739,59.366599,601.963274,146.562356,140.399994,51.578216,2.633401,4.412321,62.0,-6.934803,140.399994,0
12005,29.091299,60.463238,679.524404,136.653616,140.399994,50.474077,1.536762,3.742922,62.0,-2.361637,140.399994,0
12006,26.692466,59.308527,601.825201,157.848585,140.399994,51.421173,2.691473,4.562934,62.0,-7.244026,140.399994,0
12007,28.122638,59.329402,612.676073,148.789828,140.399994,51.545893,2.670598,4.952065,62.0,-7.132092,140.399994,0


## 标准化

In [88]:
df_zscore = df_1.copy()
df_zscores = (df_zscore - df_zscore.mean()) / df_zscore.std()

In [92]:
df_zscores['done'] = 0
df_zscores


,feed_c,out_c,feed_f,out_f,out_f_setting,pressure,error,sum_error,out_c_setting,reward,out_f_setting,done
0,-1.129065,2.047654,0.250706,-4.118932,-0.479830,-0.912333,-3.037907,-1.605876,-2.619151,-3.931575,-0.479830,0
1,-1.152532,2.929882,0.283615,-3.813948,-0.098412,-1.668567,-3.449168,-2.545292,-2.619151,-5.292574,-0.098412,0
2,-0.839948,1.494627,0.309826,-0.257629,-0.390647,-0.001681,-2.780106,-2.677018,-2.619151,-3.170421,-0.390647,0
3,-1.135378,1.492982,0.395985,-0.440789,-0.564235,-0.837678,-2.779340,-2.742494,-2.619151,-3.168263,-0.564235,0
4,-0.690787,1.529730,0.451222,-0.849897,-0.629530,-0.758434,-2.796470,-2.783886,-2.619151,-3.216627,-0.629530,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12004,0.807210,-1.008690,0.316317,0.142004,-0.132653,0.319841,0.385087,0.302843,-0.107018,0.396241,-0.132653,0
12005,0.276365,-0.538607,0.665587,-0.285877,-0.132653,0.176807,0.165952,0.235272,-0.107018,0.493633,-0.132653,0
12006,-0.047703,-1.033583,0.315695,0.629369,-0.132653,0.299497,0.396691,0.318047,-0.107018,0.389656,-0.132653,0
12007,0.145505,-1.024635,0.364558,0.238192,-0.132653,0.315654,0.392520,0.357326,-0.107018,0.392040,-0.132653,0


## 划分验证集和训练集（差集）

In [102]:
df_zscores_eval = df_zscores.sample(3000, axis=0, random_state=0)

In [103]:
df_zscores_eval

,feed_c,out_c,feed_f,out_f,out_f_setting,pressure,error,sum_error,out_c_setting,reward,out_f_setting,done
10600,-1.083354,-1.471835,-2.391384,0.592447,0.803771,-0.672516,0.600988,0.415707,-0.107018,0.250191,0.803771,0
6879,0.078736,1.427261,0.541327,-0.995686,-1.107298,0.710353,-0.650549,-0.471932,0.018589,0.405520,-1.107298,0
7542,-0.264393,-0.535382,0.301732,-0.246566,-0.151763,0.933893,0.264361,0.316730,0.018589,0.456233,-0.151763,0
343,0.418272,2.352443,0.697598,-1.863081,1.281539,1.861042,-3.179988,-3.202387,-2.619151,-4.381370,1.281539,0
874,0.369359,1.847117,2.007800,-1.469944,1.759306,1.274865,-2.944424,-2.622775,-2.619151,-3.647375,1.759306,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6358,1.140342,1.063299,0.542386,0.368674,0.326004,1.028793,-0.480884,-0.475734,0.018589,0.482362,0.326004,0
3066,-0.214821,1.990715,-0.177610,-0.412781,0.564888,0.301405,-0.313737,-0.380947,0.772229,0.528038,0.564888,0
791,-0.692030,0.985481,-0.194951,2.214290,1.759306,1.546080,-2.542762,-2.631542,-2.619151,-2.532343,1.759306,0
6616,0.327042,0.937335,-0.151285,1.919497,1.520422,0.687279,-0.422164,-0.504618,0.018589,0.501804,1.520422,0


In [104]:
df_zscores_train = df_zscores.append(df_zscores_eval)
df_zscores_train = df_zscores_train.append(df_zscores_eval)
df_zscores_train = df_zscores_train.drop_duplicates(subset=df_zscores_train.columns, keep=False)

In [105]:
df_zscores_train

,feed_c,out_c,feed_f,out_f,out_f_setting,pressure,error,sum_error,out_c_setting,reward,out_f_setting,done
0,-1.129065,2.047654,0.250706,-4.118932,-0.479830,-0.912333,-3.037907,-1.605876,-2.619151,-3.931575,-0.479830,0
1,-1.152532,2.929882,0.283615,-3.813948,-0.098412,-1.668567,-3.449168,-2.545292,-2.619151,-5.292574,-0.098412,0
2,-0.839948,1.494627,0.309826,-0.257629,-0.390647,-0.001681,-2.780106,-2.677018,-2.619151,-3.170421,-0.390647,0
3,-1.135378,1.492982,0.395985,-0.440789,-0.564235,-0.837678,-2.779340,-2.742494,-2.619151,-3.168263,-0.564235,0
4,-0.690787,1.529730,0.451222,-0.849897,-0.629530,-0.758434,-2.796470,-2.783886,-2.619151,-3.216627,-0.629530,0
...,...,...,...,...,...,...,...,...,...,...,...,...
12002,0.292517,-0.950493,0.332525,0.143972,-0.132653,0.227488,0.357958,0.300764,-0.107018,0.411077,-0.132653,0
12003,0.443388,-0.463684,-0.045329,-0.689134,-0.132653,0.259802,0.131026,0.216590,-0.107018,0.504423,-0.132653,0
12005,0.276365,-0.538607,0.665587,-0.285877,-0.132653,0.176807,0.165952,0.235272,-0.107018,0.493633,-0.132653,0
12006,-0.047703,-1.033583,0.315695,0.629369,-0.132653,0.299497,0.396691,0.318047,-0.107018,0.389656,-0.132653,0


In [107]:
data_train = df_zscores_train.values
data_eval = df_zscores_eval.values

In [118]:
df_zscores_train.loc[:, "out_f_setting"].max()

out_f_setting    1.759306
out_f_setting    1.759306
dtype: float64

In [97]:
state_dim = 8
action_dim = 1

In [108]:
replay_buffer_train = ReplayBuffer(state_dim, action_dim, device="cpu")
replay_buffer_eval = ReplayBuffer(state_dim, action_dim ,device="cpu")

In [109]:
# 训练集
for t in range(len(data_train[:-1])):
    state = data[t][0:-4]
    action = data[t][-2]
    next_state = data[t+1][0:-4]
    reward = data[t+1][-3]
    done = data[t][-1]
    replay_buffer_train.add(state, action, next_state, reward, done)
# 验证集
for t in range(len(data_eval[:-1])):
    state = data[t][0:-4]
    action = data[t][-2]
    next_state = data[t+1][0:-4]
    reward = data[t+1][-3]
    done = data[t][-1]
    replay_buffer_eval.add(state, action, next_state, reward, done)

In [110]:
buffer_name = "replay_buffer"
replay_buffer_train.save(f"./buffers/{buffer_name}_train")
replay_buffer_eval.save(f"./buffers/{buffer_name}_eval")